In [ ]:
import json
import pandas as pd
import numpy as np
import os
from pathlib import Path
import laion_clap
import torch
from sklearn.metrics.pairwise import cosine_similarity
from tqdm import tqdm
import warnings
from collections import Counter
import math
warnings.filterwarnings('ignore')

print("Libraries imported successfully!")


In [ ]:
# Load input/ground truth pairs
with open('data/input_ground_truth_pairs.json', 'r') as f:
    input_gt_pairs = json.load(f)

print(f"Loaded {len(input_gt_pairs)} sound clips with input/ground truth pairs")
print(f"Example entry: {input_gt_pairs[0]}")


In [ ]:
# Load metadata
metadata_df = pd.read_csv('data/BSD10k/BSD10K_metadata_filtered.csv')
print(f"Loaded metadata for {len(metadata_df)} sound clips")
print(metadata_df.head())


In [ ]:
# Load tagset
with open('data/tagset_clap_normalized_hyphen_unique.txt', 'r') as f:
    clap_tags = [line.strip() for line in f.readlines()]

print(f"Loaded {len(clap_tags)} unique CLAP tags")
print(f"First 10 tags: {clap_tags[:10]}")


In [ ]:
# Compute document frequency from metadata
print("Computing document frequency scores...")

# Count tag occurrences across all documents
tag_document_counts = Counter()
total_documents = len(metadata_df)

for _, row in tqdm(metadata_df.iterrows(), total=len(metadata_df), desc="Processing documents"):
    tags = row['tags'].split(',')
    tags = [tag.strip().lower() for tag in tags]  # Normalize to lowercase
    
    # Count unique tags in this document
    unique_tags = set(tags)
    for tag in unique_tags:
        tag_document_counts[tag] += 1

print(f"Found {len(tag_document_counts)} unique tags in the dataset")
print(f"Total documents: {total_documents}")

# Show top 10 most frequent tags
print("\nTop 10 most frequent tags:")
for tag, count in tag_document_counts.most_common(10):
    percentage = (count / total_documents) * 100
    print(f"{tag}: {count} documents ({percentage:.1f}%)")


In [ ]:
# Create DF mapping for CLAP tags
print("Creating DF mapping for CLAP tags...")

# Alpha parameter for DF weighting (tunable)
ALPHA = 0.7

# Create DF scores for each CLAP tag
tag_df_scores = {}
matched_tags = 0
unmatched_tags = 0

for tag in clap_tags:
    tag_lower = tag.lower()
    
    if tag_lower in tag_document_counts:
        df_count = tag_document_counts[tag_lower]
        # Compute normalized DF score with alpha weighting
        # Formula: (1 - alpha) + alpha * (log(1 + df) / log(1 + N))
        # This interpolates between no DF weighting and normalized DF weighting
        normalized_df = math.log(1 + df_count) / math.log(1 + total_documents)
        df_score = (1 - ALPHA) + ALPHA * normalized_df
        tag_df_scores[tag] = df_score
        matched_tags += 1
    else:
        # For tags not found in dataset, use minimum DF (df=0)
        # Formula: (1 - alpha) + alpha * (log(1 + 0) / log(1 + N)) = (1 - alpha)
        default_df_score = 1 - ALPHA  # Minimum possible weight
        tag_df_scores[tag] = default_df_score
        unmatched_tags += 1

print(f"\nDF Mapping Results:")
print(f"Matched tags: {matched_tags}")
print(f"Unmatched tags: {unmatched_tags}")
print(f"Alpha parameter: {ALPHA}")

# Show DF score statistics
df_scores_values = list(tag_df_scores.values())
print(f"\nDF Score Statistics:")
print(f"Min DF score: {min(df_scores_values):.4f}")
print(f"Max DF score: {max(df_scores_values):.4f}")
print(f"Mean DF score: {np.mean(df_scores_values):.4f}")
print(f"Std DF score: {np.std(df_scores_values):.4f}")

# Show examples of high and low DF tags
sorted_df_tags = sorted(tag_df_scores.items(), key=lambda x: x[1], reverse=True)
print(f"\nTop 10 highest DF score tags:")
for tag, score in sorted_df_tags[:10]:
    print(f"{tag}: {score:.4f}")

print(f"\nBottom 10 lowest DF score tags:")
for tag, score in sorted_df_tags[-10:]:
    print(f"{tag}: {score:.4f}")


In [ ]:
# Load pre-computed tag embeddings (hyphen-normalized)
print("Loading pre-computed tag embeddings...")

tags_embeddings_dir = Path('data/BSD10k/embeddings/tags')
embeddings_path = tags_embeddings_dir / 'tag_embeddings_hyphen.npy'

# Load embeddings array
tag_embeddings_array = np.load(embeddings_path)
print(f"Loaded tag embeddings with shape: {tag_embeddings_array.shape}")

# The embeddings were saved in the same order as clap_tags
# So we can use the clap_tags list as the tag names
if len(clap_tags) != tag_embeddings_array.shape[0]:
    print(f"WARNING: Mismatch between clap_tags ({len(clap_tags)}) and embeddings ({tag_embeddings_array.shape[0]})")
    # Use the smaller size to avoid index errors
    num_tags = min(len(clap_tags), tag_embeddings_array.shape[0])
    tag_names_list = clap_tags[:num_tags]
    tag_embeddings_array = tag_embeddings_array[:num_tags]
else:
    tag_names_list = clap_tags
    num_tags = len(clap_tags)

print(f"Number of tags: {num_tags}")
print(f"Embedding dimension: {tag_embeddings_array.shape[2]}")
print(f"Sentence variants per tag: {tag_embeddings_array.shape[1]}")

# Convert to dictionary format (same as original)
tag_embeddings = {}
for i, tag in enumerate(tag_names_list):
    tag_embeddings[tag] = tag_embeddings_array[i]

print(f"\nConverted to dictionary format with {len(tag_embeddings)} tags")
print(f"Example tag embedding shape: {list(tag_embeddings.values())[0].shape}")

# Verify that our tag_df_scores and tag_embeddings have matching tags
matching_tags = set(tag_df_scores.keys()) & set(tag_embeddings.keys())
print(f"\nTags with both DF scores and embeddings: {len(matching_tags)}")
print(f"Tags with DF scores only: {len(set(tag_df_scores.keys()) - set(tag_embeddings.keys()))}")
print(f"Tags with embeddings only: {len(set(tag_embeddings.keys()) - set(tag_df_scores.keys()))}")


In [ ]:
# Check available audio embeddings
embeddings_dir = Path('data/BSD10k/embeddings/clap')
available_embeddings = set([int(f.stem) for f in embeddings_dir.glob('*.npy')])

print(f"Found {len(available_embeddings)} audio embeddings")

# Filter input_gt_pairs to only include clips with available embeddings
filtered_pairs = [pair for pair in input_gt_pairs if pair['sound_id'] in available_embeddings]

print(f"Using {len(filtered_pairs)} clips with both ground truth and audio embeddings")


In [ ]:
# Load audio embeddings for the filtered clips
print("Loading audio embeddings...")

audio_embeddings = {}

for pair in tqdm(filtered_pairs):
    sound_id = pair['sound_id']
    embedding_path = embeddings_dir / f"{sound_id}.npy"
    
    if embedding_path.exists():
        audio_embeddings[sound_id] = np.load(embedding_path)

print(f"Loaded audio embeddings for {len(audio_embeddings)} sound clips")
if audio_embeddings:
    sample_embedding = list(audio_embeddings.values())[0]
    print(f"Audio embedding dimension: {sample_embedding.shape}")


In [ ]:
def get_tag_recommendations_df_weighted(audio_embedding, tag_embeddings, tag_df_scores, top_k=10):
    """
    Get top-k tag recommendations for a given audio embedding with DF weighting.
    For each tag, we take the maximum similarity among its 4 sentence variants,
    then multiply by the DF weight.
    
    Final score = cosine_similarity × df_weight
    where df_weight = (1 - alpha) + alpha * (log(1 + df) / log(1 + N))
    """
    tag_similarities = {}
    
    for tag, text_embeds in tag_embeddings.items():
        # Compute cosine similarity between audio and all text variants
        similarities = cosine_similarity([audio_embedding], text_embeds)[0]
        # Take the maximum similarity among the 4 variants
        base_similarity = np.max(similarities)
        
        # Apply DF weighting
        df_weight = tag_df_scores.get(tag, 1.0)  # Default to 1.0 if tag not found
        weighted_similarity = base_similarity * df_weight
        
        tag_similarities[tag] = {
            'base_similarity': base_similarity,
            'df_weight': df_weight,
            'weighted_similarity': weighted_similarity
        }
    
    # Sort tags by weighted similarity and get top-k
    sorted_tags = sorted(tag_similarities.items(), key=lambda x: x[1]['weighted_similarity'], reverse=True)
    
    top_tags = [tag for tag, _ in sorted_tags[:top_k]]
    top_scores = [scores['weighted_similarity'] for _, scores in sorted_tags[:top_k]]
    top_base_scores = [scores['base_similarity'] for _, scores in sorted_tags[:top_k]]
    top_df_weights = [scores['df_weight'] for _, scores in sorted_tags[:top_k]]
    
    return top_tags, top_scores, top_base_scores, top_df_weights

# Test the function
test_sound_id = filtered_pairs[0]['sound_id']
test_audio_embedding = audio_embeddings[test_sound_id]
test_recommendations, test_scores, test_base_scores, test_df_weights = get_tag_recommendations_df_weighted(
    test_audio_embedding, tag_embeddings, tag_df_scores, top_k=5
)

print(f"Test DF-weighted recommendations for sound {test_sound_id}:")
for i, (tag, score, base_score, df_weight) in enumerate(zip(test_recommendations, test_scores, test_base_scores, test_df_weights), 1):
    print(f"{i}. {tag} (weighted: {score:.4f}, base: {base_score:.4f}, DF weight: {df_weight:.4f})")


In [ ]:
# Generate DF-weighted recommendations for all clips
print("Generating DF-weighted recommendations for all clips...")

results = []

for pair in tqdm(filtered_pairs):
    sound_id = pair['sound_id']
    
    if sound_id not in audio_embeddings:
        continue
    
    # Get audio embedding
    audio_embedding = audio_embeddings[sound_id]
    
    # Get top 10 DF-weighted recommendations
    predicted_tags, prediction_scores, base_scores, df_weights = get_tag_recommendations_df_weighted(
        audio_embedding, tag_embeddings, tag_df_scores, top_k=10
    )
    
    # Normalize ground truth tags to lowercase for comparison
    ground_truth_tags = [tag.lower() for tag in pair['ground_truth_tags']]
    predicted_tags_lower = [tag.lower() for tag in predicted_tags]
    
    # Calculate hits (intersection of predicted and ground truth)
    hits = list(set(predicted_tags_lower) & set(ground_truth_tags))
    
    # Calculate metrics
    precision = len(hits) / 10.0
    recall = len(hits) / len(ground_truth_tags) if ground_truth_tags else 0.0
    f1 = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0.0
    
    result = {
        'sound_id': sound_id,
        'title': pair['title'],
        'ground_truth_tags': ground_truth_tags,
        'predicted_tags': predicted_tags,
        'prediction_scores': prediction_scores,
        'base_scores': base_scores,
        'df_weights': df_weights,
        'hits': hits,
        'num_hits': len(hits),
        'precision_at_10': precision,
        'recall': recall,
        'f1_score': f1
    }
    
    results.append(result)

print(f"Generated DF-weighted recommendations for {len(results)} clips")


In [ ]:
# Calculate overall metrics
total_hits = sum(result['num_hits'] for result in results)
total_predictions = len(results) * 10
total_ground_truth = sum(len(result['ground_truth_tags']) for result in results)

avg_precision_at_10 = np.mean([result['precision_at_10'] for result in results])
avg_recall = np.mean([result['recall'] for result in results])
avg_hits_per_clip = np.mean([result['num_hits'] for result in results])

# Calculate F1 scores
overall_precision = total_hits / total_predictions
overall_recall = total_hits / total_ground_truth
overall_f1 = 2 * (overall_precision * overall_recall) / (overall_precision + overall_recall) if (overall_precision + overall_recall) > 0 else 0

# Calculate average F1 score per clip
f1_scores = []
for result in results:
    precision = result['precision_at_10']
    recall = result['recall']
    f1 = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0
    f1_scores.append(f1)
avg_f1 = np.mean(f1_scores)

print(f"=== CLAP DF-Weighted Tag Recommendation Results (Alpha = {ALPHA}) ===")
print(f"Number of clips evaluated: {len(results)}")
print(f"Total hits: {total_hits}")
print(f"Total predictions: {total_predictions}")
print(f"Total ground truth tags: {total_ground_truth}")
print()
print(f"Average Precision@10: {avg_precision_at_10:.4f}")
print(f"Average Recall: {avg_recall:.4f}")
print(f"Average F1 Score: {avg_f1:.4f}")
print(f"Average hits per clip: {avg_hits_per_clip:.2f}")
print()
print(f"Overall Precision: {overall_precision:.4f}")
print(f"Overall Recall: {overall_recall:.4f}")
print(f"Overall F1 Score: {overall_f1:.4f}")


In [ ]:
# Show some example results with DF analysis
print("=== Example DF-Weighted Results ===")

# Sort by number of hits (best first)
results_sorted = sorted(results, key=lambda x: x['num_hits'], reverse=True)

for i, result in enumerate(results_sorted[:5]):
    print(f"\n--- Example {i+1} (Sound ID: {result['sound_id']}) ---")
    print(f"Title: {result['title']}")
    print(f"Ground Truth Tags: {result['ground_truth_tags']}")
    print(f"Predicted Tags (with DF weights):")
    for j, (tag, score, base_score, df_weight) in enumerate(zip(
        result['predicted_tags'], result['prediction_scores'], 
        result['base_scores'], result['df_weights']
    ), 1):
        hit_marker = "★" if tag.lower() in result['hits'] else " "
        print(f"  {j:2d}.{hit_marker} {tag:<20} (W:{score:.4f}, B:{base_score:.4f}, DF:{df_weight:.3f})")
    print(f"Hits: {result['hits']} ({result['num_hits']} hits)")
    print(f"Precision@10: {result['precision_at_10']:.3f}, Recall: {result['recall']:.3f}, F1: {result['f1_score']:.3f}")


In [ ]:
# Distribution of hits
hits_distribution = {}
for result in results:
    num_hits = result['num_hits']
    hits_distribution[num_hits] = hits_distribution.get(num_hits, 0) + 1

print("\n=== Distribution of Hits ===")
for hits in sorted(hits_distribution.keys()):
    count = hits_distribution[hits]
    percentage = count / len(results) * 100
    print(f"{hits} hits: {count} clips ({percentage:.1f}%)")


In [ ]:
# Convert numpy values to Python types for JSON serialization
def convert_numpy_types(obj):
    """Recursively convert numpy types to Python types"""
    if isinstance(obj, np.integer):
        return int(obj)
    elif isinstance(obj, np.floating):
        return float(obj)
    elif isinstance(obj, np.ndarray):
        return obj.tolist()
    elif isinstance(obj, dict):
        return {key: convert_numpy_types(value) for key, value in obj.items()}
    elif isinstance(obj, list):
        return [convert_numpy_types(item) for item in obj]
    else:
        return obj

# Convert results to be JSON serializable
json_safe_results = convert_numpy_types(results)

# Save results
output_file = f'eval/clap_baseline_df_alpha{ALPHA}_results.json'
os.makedirs('eval', exist_ok=True)

# Prepare summary
summary = {
    'model': f'CLAP DF-Weighted (Alpha={ALPHA})',
    'alpha': float(ALPHA),
    'num_clips': len(results),
    'total_hits': int(total_hits),
    'total_predictions': int(total_predictions),
    'total_ground_truth': int(total_ground_truth),
    'avg_precision_at_10': float(avg_precision_at_10),
    'avg_recall': float(avg_recall),
    'avg_f1_score': float(avg_f1),
    'avg_hits_per_clip': float(avg_hits_per_clip),
    'overall_precision': float(overall_precision),
    'overall_recall': float(overall_recall),
    'overall_f1_score': float(overall_f1),
    'hits_distribution': convert_numpy_types(hits_distribution),
    'df_stats': {
        'matched_tags': matched_tags,
        'unmatched_tags': unmatched_tags,
        'min_df_score': float(min(df_scores_values)),
        'max_df_score': float(max(df_scores_values)),
        'mean_df_score': float(np.mean(df_scores_values)),
        'std_df_score': float(np.std(df_scores_values))
    }
}

# Save detailed results
with open(output_file, 'w') as f:
    json.dump({
        'summary': summary,
        'detailed_results': json_safe_results
    }, f, indent=2)

print(f"Results saved to {output_file}")

# Also save summary only
summary_file = f'eval/clap_baseline_df_alpha{ALPHA}_summary.json'
with open(summary_file, 'w') as f:
    json.dump(summary, f, indent=2)

print(f"Summary saved to {summary_file}")
